test for doesthedogdie API

Imports

In [ ]:
import json
import re
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# # 12533 -> wall-e
# # 9593 -> fight club

# url = 'https://www.doesthedogdie.com/media/12533'
# headers = {
#     'Accept': 'application/json',
#     'X-API-KEY': '8bf1393430a063dd2b27f3699f1625bc'
# }

# response = requests.get(url, headers=headers)

# if response.status_code == 200:
#     data = response.json()
#     print(json.dumps(data, indent=4))  # Print the full response

#     # Check the structure to access the ID correctly
#     if 'id' in data:  # Check if 'id' is in the top-level response
#         movie_id = data['id']
#         print(f"Movie ID: {movie_id}")
#     else:
#         print("Key 'id' not found in the response.")
# else:
#     print(f"Error: {response.status_code} - {response.text}")

In [ ]:
# Define the API request
url = 'https://www.doesthedogdie.com/media/10016'
headers = {
    'Accept': 'application/json',
    'X-API-KEY': '8bf1393430a063dd2b27f3699f1625bc'
}

# Send the request
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()

    # Initialize list to store extracted data
    extracted_data = []

    # Iterate over the topicItemStats to collect itemName, releaseYear, and triggers
    for topic_item in data.get('topicItemStats', []):
        # Extract itemName and releaseYear from the current topic item
        clean_name = topic_item.get('cleanName', 'Unknown')
        item_name = topic_item.get('itemName', 'Unknown')
        release_year = topic_item.get('releaseYear', 'Unknown')
        
        # Initialize a list to store triggers for the current movie
        triggers = []  
        
        # Extract topic information nested inside each topicItem
        topic = topic_item.get('topic', {})
        if topic:
            # Extract 'doesName' from the topic and the corresponding yes/no counts
            does_name = topic.get('doesName', 'Unknown')
            yes_sum = topic_item.get('yesSum', 0)
            no_sum = topic_item.get('noSum', 0)

            # Append trigger information to the list of triggers
            triggers.append({
                "doesName": does_name,
                "yesSum": yes_sum,
                "noSum": no_sum
            })
        
        # Check if this movie already exists in extracted_data
        movie_exists = False
        for movie in extracted_data:
            if movie["itemName"] == item_name and movie["releaseYear"] == release_year:
                # If movie already exists, add the trigger to it
                movie["triggers"].append({
                    "doesName": does_name,
                    "yesSum": yes_sum,
                    "noSum": no_sum
                })
                movie_exists = True
                break
        
        # If movie doesn't exist, add a new entry with itemName, releaseYear, and the current trigger
        if not movie_exists:
            extracted_data.append({
                "itemName": item_name,
                "releaseYear": release_year,
                "triggers": triggers
            })

    # print extracted data
    print(json.dumps(extracted_data, indent=4))

else:
    print(f"Error: {response.status_code} - {response.text}")


Flatten data and generate a DataFrame

In [ ]:
# flaten data into a list of dictionaries for the df
flattened_data = []
for item in extracted_data:
    item_name = item["itemName"]
    release_year = item["releaseYear"]
    for trigger in item["triggers"]:
        flattened_data.append({
            "itemName": item_name,
            "releaseYear": release_year,
            "doesName": trigger["doesName"],
            "yesSum": trigger["yesSum"],
            "noSum": trigger["noSum"]
        })

df = pd.DataFrame(flattened_data)

# manually entered filename
# df.to_csv('howls-moving-castle_df.csv', index=False)

df.head()

Generate Plot with top 20 triggers.

In [ ]:
# # Assuming 'extracted_data' is the JSON response
# # If it's a JSON string, parse it into a Python object
# # If 'extracted_data' is already a Python dictionary or list, skip the following line
# # extracted_data = json.loads(extracted_data)

# # Assuming 'extracted_data' is a list of dictionaries as per your response structure

# # Initialize an empty list to store flattened data
# flattened_data = []

# # Iterate through the JSON response to extract relevant fields
# for item in extracted_data:
#     item_name = item['itemName']
#     release_year = item['releaseYear']
    
#     # Iterate through the triggers and extract 'doesName', 'yesSum', and 'noSum'
#     for trigger in item['triggers']:
#         flattened_data.append({
#             'Item Name': item_name,
#             'Release Year': release_year,
#             'Trigger': trigger['doesName'],
#             'Yes Count': trigger['yesSum'],
#             'No Count': trigger['noSum']
#         })

# # Convert the flattened data into a DataFrame
# df = pd.DataFrame(flattened_data)

# # Group the data by 'Trigger' and sum the 'Yes Count'
# trigger_yes_sum = df.groupby('Trigger')['Yes Count'].sum().reset_index()

# # Sort the triggers by 'Yes Count' in descending order
# trigger_yes_sum = trigger_yes_sum.sort_values(by='Yes Count', ascending=False)

# # Select the top 20 triggers with the highest "Yes Count"
# top_20_triggers = trigger_yes_sum.head(20)

# # Find the unique movie names for the top 20 triggers
# top_20_movie_names = df[df['Trigger'].isin(top_20_triggers['Trigger'])]['Item Name'].unique()

# # Create a dynamic title with the movie names
# movie_names_title = ", ".join(top_20_movie_names[:5]) + ("..." if len(top_20_movie_names) > 5 else "")

# # Plot the data for the top 20 triggers
# plt.figure(figsize=(10, 6))
# plt.barh(top_20_triggers['Trigger'], top_20_triggers['Yes Count'], color='lightcoral')
# plt.xlabel('Vote Count')
# plt.title(f'Top 20 Triggers in {movie_names_title}')
# plt.gca().invert_yaxis()  # Invert y-axis to show the highest value at the top
# plt.show()
